In [16]:
# Authors: Z Wojna, S Morrell
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20, 20)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['figure.max_open_warning'] = 100

import sys
import json
import os
import re
import shutil
import subprocess
import random
import PIL.Image
import cv2
import numpy as np
from collections import OrderedDict
import copy
import xmltodict
import sys
import math
import time
import PIL.ImageDraw as ImageDraw
import PIL.Image as Image
import cStringIO as StringIO
import csv
import pandas
import xlrd
import dicom
from scipy.misc import imresize
import os
import pandas as pd
import scipy.ndimage.interpolation
import math

## Prepare images_lesions_all.xlsx

In [94]:
step = 0
path = '/home/smorrell/git/Mammo/devdirs/surrey_stuff/xls/batch_files_xls_latest'  # dir containing batch_1_IMAGE.xls, batch_1_NBSS.xls, ...

for im_file in os.listdir(path):
    if '_IMAGE' not in im_file:
        continue
    print(im_file)
    xl = pd.ExcelFile(os.path.join(path, im_file))
    images_temp = xl.parse('StudyImageAttributes')  # [u'StudyImageAttributes', u'LesionMarkAttributes']
    lesions_temp = xl.parse('LesionMarkAttributes')
    if step == 0:
        images_df = images_temp
        lesions_df = lesions_temp
        step += 1
    else:
        images_df = images_df.append(images_temp)
        lesions_df = lesions_df.append(lesions_temp)
    print len(images_df), len(lesions_df)

writer = pd.ExcelWriter(os.path.join(path, 'images_lesions_all.xlsx'))
images_df.to_excel(writer, 'StudyImageAttributes')
lesions_df.to_excel(writer, 'LesionMarkAttributes')
writer.save()

batch_16_IMAGE.xls
4195 387
batch_8_IMAGE.xls
8072 798
batch_4_IMAGE.xls
9692 798
batch_7_IMAGE.xls
13599 1212
batch_3_IMAGE.xls
16849 1630
batch_5_IMAGE.xls
20701 2044
batch_6_IMAGE.xls
24468 2454
batch_18_IMAGE.xls
27775 2977
batch_19_IMAGE.xls
32300 3584
batch_31_IMAGE.xls
33944 3584
batch_12_IMAGE.xls
38215 3988
batch_11_IMAGE.xls
41803 4401
batch_14_IMAGE.xls
44999 4784
batch_9_IMAGE.xls
46602 4784
batch_1_IMAGE.xls
49302 5185
batch_10_IMAGE.xls
53341 5619
batch_13_IMAGE.xls
56430 6029
batch_22_IMAGE.xls
61632 6463
batch_23_IMAGE.xls
63851 6695
batch_2_IMAGE.xls
65421 6695
batch_21_IMAGE.xls
69251 7105
batch_17_IMAGE.xls
72163 7392
batch_15_IMAGE.xls
74559 7409
batch_32_IMAGE.xls
76243 7409
batch_20_IMAGE.xls
78377 7657


In [21]:
lesions_images = list(lesions_df.ix[:, 'ImageSOPIUID'].unique())
lesions_images = [x + '.dcm' for x in lesions_images]
print len(lesions_images)

In [166]:
lesions_df.index = range(len(lesions_df))

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            7647, 7648, 7649, 7650, 7651, 7652, 7653, 7654, 7655, 7656],
           dtype='int64', length=7657)

In [171]:
writer = pd.ExcelWriter('/home/smorrell/git/Mammo/devdirs/images_lesions_all.xlsx')
images_df.to_excel(writer, 'StudyImageAttributes')
lesions_df.to_excel(writer, 'LesionMarkAttributes')
writer.save()

## Count by manufacturer, and unique lesions

In [44]:
manuf = {}
for i in range(len(lesions_df)):
    imid = lesions_df.ix[i, u'ImageSOPIUID']
    m = images_df[images_df['ImageSOPIUID'] == imid]['Manufacturer']
    m = str(m.values)
    if m not in manuf.keys():
        manuf[m] = 1
    else:
        manuf[m] += 1

manuf

{"[u'GE MEDICAL SYSTEMS']": 290,
 "[u'HOLOGIC, Inc.']": 6351,
 "[u'Philips Digital Mammography Sweden AB']": 878,
 "[u'Philips Medical Systems']": 14,
 "[u'SIEMENS']": 124}

##### Choose image set from either lesions or normal_screening

In [ ]:
# (A) Run this for lesions
ROOT_DIR = '/home/smorrell/git/Mammo/devdirs/surrey_stuff/xls/'
xl = pd.ExcelFile(ROOT_DIR + 'images_lesions_all.xlsx')
lesions_df = xl.parse('LesionMarkAttributes')
images_df = xl.parse('StudyImageAttributes')
images_to_make = list(lesions_df.ix[:, 'ImageSOPIUID'].unique())

In [22]:
# (B) Run this for normal_screening
xl = pd.ExcelFile('/home/smorrell/Dropbox/BreastUcl/MyNotes/20180227_normal_screening.xlsx')
normal_df = xl.parse('Sheet1')
images_to_make = list(normal_df.loc[:, 'ImageSOPIUID'].unique())

In [23]:
images_to_make = [x + '.dcm' for x in images_to_make]
print len(images_to_make)
print images_to_make[:2]

19547
[u'1.2.840.113681.2229466362.959.3603083047.390.1.dcm', u'1.2.840.113681.2229466362.959.3603083047.394.1.dcm']


## Make full-res, intensity-corrected png from dicoms.
#### Warning: this cell may overwrite existing png's.

In [20]:
# path_to_png = '/home/smorrell/git/Mammo/devdirs/surrey_png_fullres'
path_to_png = '/media/smorrell/data/surrey_png'

In [183]:
# 7000 images = 18GB
for img_filename in images_to_make:
    found = False
    print img_filename,
    
    for path, subdirs, files in os.walk('/media/3tb/surrey/'):
        for filename in files:
            if filename == img_filename:
                plan = dicom.read_file(path + '/' + filename)
                im = plan.pixel_array
                
                print plan.Manufacturer,
                if 'VOILUTSequence' in dir(plan):
                    print 'applying LUT transform',
                    if type(plan.VOILUTSequence) == dicom.sequence.Sequence:
                        vls = plan.VOILUTSequence[1]
                    else:
                        vls = plan.VOILUTSequence

                    if type(vls.LUTData) == str:
                        vlsdata = range(len(vls.LUTData)/2)
                        for i in range(0, len(vls.LUTData), 2):
                            vlsdata[i/2] = int((vls.LUTData[i+1] + vls.LUTData[i]).encode('hex'), 16)
                        vlsdata = np.array(vlsdata)
                    else:
                        vlsdata = np.array(vls.LUTData)

                    pmin = vls.LUTDescriptor[1]
                    clipdata = np.clip(plan.pixel_array, pmin, pmin + len(vlsdata) - 1)
                    im = (vlsdata[clipdata - pmin]).astype(np.uint16)
                    
                elif plan.Manufacturer == 'Philips Digital Mammography Sweden AB':
                    print 'applying window sigmoid transform',
                    center = float(plan.WindowCenter[0])
                    width = float(plan.WindowWidth[0])
                    assert plan.VOILUTFunction == 'SIGMOID'
                    im = 4096 / (1 + np.exp(-4 * (im - center) / width))
        
                im = 255.0 * (im - np.min(im)).astype(float) / np.float(np.max(im) - np.min(im))
                im = im.astype(int)
                if np.mean(im) > 150:
                    im = 255 - im
                cv2.imwrite(path_to_png + '/' + filename[:-3] + 'png', im)
                found = True
                print('found')
    if not found:
        print('-------------------not found-------------------')

1.2.840.113681.2229460924.931.3527911846.43.1.dcm
HOLOGIC, Inc.
1.2.840.113681.2229460924.931.3527911846.45.1.dcm
HOLOGIC, Inc.
1.2.840.113681.181135466.1313654044.1936.3437.dcm
HOLOGIC, Inc.
1.2.840.113681.181135466.1313654044.1936.3441.dcm
HOLOGIC, Inc.
1.2.840.113681.2230565232.2126.3492832864.501.1.dcm
HOLOGIC, Inc.
1.2.840.113681.2230565232.2126.3492832864.505.1.dcm
HOLOGIC, Inc.
1.2.840.113681.2229458136.951.3525926746.260.1.dcm
HOLOGIC, Inc.
1.2.840.113681.2229458136.951.3525926746.264.1.dcm
HOLOGIC, Inc.
1.2.840.113681.181135466.1321430698.1096.1813.dcm
HOLOGIC, Inc.
1.2.840.113681.181135466.1321430698.1096.1809.dcm
HOLOGIC, Inc.
1.3.46.670589.30.1.3.1.146450984119.1308914481640.2.dcm
Philips Medical Systems
1.3.46.670589.30.1.3.1.146450984119.1308914497562.2.dcm
Philips Medical Systems
1.3.46.670589.30.1.3.1.146450984119.1308565122531.2.dcm
Philips Medical Systems
1.3.46.670589.30.1.3.1.146450984119.1308565135812.2.dcm
Philips Medical Systems
1.3.46.670589.30.1.3.1.14645098411

## Make annotations file

In [ ]:
# Run for (A) & (B) for both lesion files
# Read images_lesions_all.xlsx
ROOT_DIR = '/home/smorrell/git/Mammo/devdirs/'
xl = pd.ExcelFile(ROOT_DIR + 'images_lesions_all.xlsx')

In [35]:
# Initialise
xml_annotations = {'type': 'instances', 
                   'annotations': [], 
                   'images': [], 
                   'categories': [{'id': 1, 'name': 'MALIGNANT'}, {'id': 2, 'name': 'BENIGN'}, 
                                  {'id': 3, 'name': 'NORMAL'}]
                  }
IMAGE_MAX_DIM = 3328
ImageSOPIUIDtoID = {}
# Fields are renamed:
data_added = ['ClientID', 'StudyIUID', 'ViewPosition', 'ImageLaterality']
renamed_to = ['subjectId', 'examIndex', 'view', 'laterality']

#### (A) normal screening
Run both 'images_to_make' cells above.

In [36]:
tsv_image_metadata = xl.parse('StudyImageAttributes')

In [39]:
image_id = 100000
for i, row in enumerate(images_to_make):
    if row[:-4] == '1.2.840.113681.2229466362.959.3603083047.390.1.xxxx':  # omit these 
        print 'omitting'
        continue
    print i, row
    
    # Load image and data
    image = cv2.imread(path_to_png + '/' +  row[:-3] + 'png')
    
    study_image_row = tsv_image_metadata.loc[tsv_image_metadata['ImageSOPIUID'] == row[:-4]]
    assert len(study_image_row) == 1, ('Error: < 1 or > 1 record for ', row)
    image_meta = {}
    for source_field, dest_field in zip(data_added, renamed_to):
        image_meta[dest_field] = study_image_row[source_field].to_string(index=False)
    image_meta['file_name'] = row[:-4] + '.png'
    image_meta['height'] = image.shape[0]
    image_meta['width'] = image.shape[1]
    image_meta['id'] = image_id
    xml_annotations['images'] += [image_meta]
    image_id += 1
print xml_annotations
with open('/home/smorrell/git/zztop/Deformable-ConvNets/data/coco/annotations/instances_normals.json', 'w') as fp:
    json.dump(xml_annotations, fp, indent=4)

0 1.2.840.113681.2229466362.959.3603083047.390.1.dcm
1 1.2.840.113681.2229466362.959.3603083047.394.1.dcm
2 1.2.840.113681.2229461008.961.3601270080.135.1.dcm
3 1.2.840.113681.2229461008.961.3601270080.139.1.dcm
4 1.2.840.113681.2229460924.932.3602305285.95.1.dcm
5 1.2.840.113681.2229460924.932.3602305285.93.1.dcm
6 1.2.840.113619.2.227.2079195738.2287150311093755.11.dcm
7 1.2.840.113619.2.227.2079195738.2287150311093733.9.dcm
8 1.2.840.113619.2.227.2079195738.2287150311093524.5.dcm
9 1.2.840.113619.2.227.2079195738.2287150311093657.7.dcm
10 1.2.840.113681.2229461008.941.3603428967.452.1.dcm
11 1.2.840.113681.2229461008.941.3603428967.450.1.dcm
12 1.2.840.113681.2229461008.941.3603428967.454.1.dcm
13 1.2.840.113681.2229460924.952.3601268442.419.1.dcm
14 1.2.840.113681.2229460924.952.3601268442.417.1.dcm
15 1.2.840.113681.2229460924.952.3601268442.423.1.dcm
16 1.2.840.113681.2229466322.1997.3601701021.367.1.dcm
17 1.2.840.113681.2229466322.1997.3601701021.365.1.dcm
18 1.2.840.113619.2.2

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#### (B) lesions
TODO: Add metadata from StudyImageAttributes

In [12]:
# Load lesion data
tsv_annotations = xl.parse('LesionMarkAttributesWithImageDim')
image_id = 1
anno_id = 1
annos = []

In [ ]:
DATA_SET = 1

for i in range(0, len(tsv_annotations)):
    row = tsv_annotations.ix[i, :]
    print i
    
    # Omit some images, check status
    if row['batch'] != 1:
        continue
    # image of some metal part, no breast here
    if row['ImageSOPIUID'] == '1.2.840.113619.2.227.2079161146212.2295131007153007.103':
        continue
    if row['Status'] == 'B1':
        continue
    assert row['Status'] in ['Malignant', 'Benign', 'B2', 'B3', 'B4', 'B5']
    
    # Check box dimensions
    box_width = row['Width']
    box_height = row['Height']
    if box_width <= 0 or box_height <= 0:
        print 'box width/heigth <= 0'
        continue 
    image = cv2.imread(ROOT_DIR + 'surrey_png_fullres/' +  row['ImageSOPIUID'] + '.png')
    assert box_width == row['X2'] - row['X1']
    if not row['X2'] <= image.shape[1]:
        print row['X2'], image.shape[1]
        print 'X2 more than width'
        print row['ImageSOPIUID']
        row['X2'] = image.shape[1]
    assert box_height == row['Y2'] - row['Y1']
    if not row['Y2'] <= image.shape[0]:
        print row['Y2'], image.shape[0] 
        print 'Y2 more than Height'
        print row['ImageSOPIUID']
        row['Y2'] = image.shape[0]
    
    # Rescale boxes
    scale = float(IMAGE_MAX_DIM) / float(np.max(image.shape))
    image = imresize(image, size = scale).astype(np.uint8)
    box_width = int(scale * box_width)
    box_height = int(scale * box_height)
    row['X1'] = int(scale * row['X1'])
    row['Y1'] = int(scale * row['Y1'])
    row['X2'] = int(scale * row['X2'])
    row['Y2'] = int(scale * row['Y2'])

    # Make Augmentations
    image_before = image
    for rot in range(0, 4):  # 1: 90 deg anticlockwise. Top left: x=0, y=0
        image = copy.deepcopy(image_before)
        image = scipy.ndimage.interpolation.rotate(image, angle=90*rot)
        # image = np.rot90(image, k = rot)  # difference with 4gpu version
        # cv2.imwrite('temp.png', image)
        # image = cv2.imread('temp.png')
        im_name = row['ImageSOPIUID'] + '_rot' + str(rot)
        if im_name not in ImageSOPIUIDtoID:
            ImageSOPIUIDtoID[im_name] = image_id
            image_id += 1
            xml_annotations['images'] += [
                {'file_name': im_name + '.png',
                 'height': image.shape[0],
                 'width': image.shape[1],
                 'id': ImageSOPIUIDtoID[im_name]
                }
            ]

        if rot:  #  == 1 or rot == 3:
            box_width, box_height = box_height, box_width
        
        x = (row['X1'] + row['X2']) / 2
        y = (row['Y1'] + row['Y2']) / 2
        if rot == 0:
            xx, yy = x, y
        if rot == 1:  # 90 deg anticlockwise
            xx = y
            yy = image_before.shape[1] - x
        if rot == 2:
            xx = image_before.shape[1] - x
            yy = image_before.shape[0] - y
        if rot == 3:
            xx = image_before.shape[0] - y
            yy = x
        x, y = xx, yy
        
        anno = {}
        anno['bbox'] = [x - box_width / 2, y - box_height / 2, box_width, box_height]
        anno['area'] = box_width * box_height
        anno['iscrowd'] = 0
        anno['image_id'] = ImageSOPIUIDtoID[im_name]
        anno['category_id'] = 1 if row['Status'] in ['Malignant', 'B4', 'B5'] else 2
        anno['id'] = anno_id
        anno_id += 1
        xml_annotations['annotations'] += [anno]

        cv2.imwrite(ROOT_DIR + 'surrey_png_for_training/' + im_name + '.png', image)
        cv2.rectangle(image, (anno['bbox'][0], anno['bbox'][1]), 
                     (anno['bbox'][0] + anno['bbox'][2], anno['bbox'][1] + anno['bbox'][3]), 
                     (0, 256 * int(anno['category_id'] == 2), 256 * int(anno['category_id'] == 1)),
                     4)
        cv2.imwrite(ROOT_DIR + 'surrey_png_annotated/' + im_name + '.png', image)

    #if i == 2:
    #    break
    
with open(ROOT_DIR + 'mammo.json', 'w') as fp:
    json.dump(xml_annotations, fp, indent=4)

0
1
2
3
4
5
6


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

7
8
9
10
11
